# CityBikes

Send a request to CityBikes for the city of your choice. 

In [6]:
import pandas as pd
import os # use this to access your environment variables
import requests # this will be used to call the APIs
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [7]:
# fields_list = ['1', '2', '3']

# url = "http://api.citybik.es/v2/networks"
    
# if len(fields_list) > 0:
#     url = url + "?fields="
#     for field in fields_list:
#         url = url + field + ","

# url = url[:-1]


In [8]:
def get_citybikes(networkid=None, fields_list=[]):
    url = "https://api.citybik.es/v2/networks"
    
    if networkid is not None:
        url = url + "/" + networkid
    else:
        pass

    if len(fields_list) > 0:
        url = url + "?fields="
        for field in fields_list:
            url = url + field + ","
        url = url[:-1]
    else:
        pass

    response = requests.get(url)

    return response

In [9]:
#returns a response of the networks, which we can then use to find the Toronto networkid
get_networks = get_citybikes()
networks_json = get_networks.json()

In [10]:
#create a df of the network information
networks_df = pd.DataFrame(networks_json['networks'])

In [11]:
#creates a DF of all the location data from the 'networks' endpoint
location_df = pd.DataFrame(list(networks_df['location']))

In [14]:
#finds the location of the specified city - in this case 'toronto'
tor_search = location_df['city'].str.contains('toronto', case=False)

In [15]:
#retrieve the id for toronto as single value variable
tor_id = networks_df.loc[tor_search, ['id']].iloc[0].values[0]

In [16]:
#call api with tor_id variable to get toronto information and store as json
tor_json = get_citybikes(tor_id).json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [17]:
tor_stations = pd.DataFrame(tor_json['network']['stations']).drop(['extra'], axis=1)

In [18]:
#format time column; convert to local timezone
tor_stations['timestamp'] = pd.to_datetime(tor_stations['timestamp'], utc=True, format='ISO8601').dt.tz_convert('US/Eastern')

#confirm id's are unique (rows = nunique for id)
print(tor_stations.shape)
print(tor_stations['id'].nunique())

(751, 7)
751


Put your parsed results into a DataFrame.

In [19]:
#show the df
tor_stations

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp
0,2,16,fb337bbed72e2be090071e199899b2be,43.665269,-79.319796,Queen St E / Woodward Ave,2023-11-17 15:55:44.074000-05:00
1,13,1,4ff88d5880e71aa40d34cfe5d09b0ca7,43.671420,-79.445947,Primrose Ave / Davenport Rd,2023-11-17 15:55:44.075000-05:00
2,14,8,a09c67c0b419654d907c9134b108e328,43.666224,-79.317693,Queen St E / Rhodes Ave,2023-11-17 15:55:44.076000-05:00
3,4,31,d6a9daee68070a8b106cfb598d81308c,43.653236,-79.376716,Bond St / Queen St E,2023-11-17 15:55:44.101000-05:00
4,9,17,8f8af40d9388c8a3962559e8681d3db7,43.663722,-79.380288,Church St / Alexander St,2023-11-17 15:55:44.104000-05:00
...,...,...,...,...,...,...,...
746,15,0,3654ed1bd8516fddd2364502084cd01b,43.748454,-79.550404,Lindylou Rd / Lanyard Rd,2023-11-17 15:55:44.184000-05:00
747,15,0,3f09d8a338e4501752365339f106a1ff,43.763770,-79.488092,Tangiers Rd / Finch Ave W,2023-11-17 15:55:44.186000-05:00
748,16,0,8402cda81d70fbc893e5337366df8d58,43.661803,-79.389682,Queen's Park Cres E / Grosvenor St - SMART,2023-11-17 15:55:45.560000-05:00
749,11,0,eb00d171668cedf07334d8504a1dbd79,43.727498,-79.327142,Wynford Heights Cres / Concorde Pl,2023-11-17 15:55:44.185000-05:00


In [104]:
#save tor stations as csv
tor_stations = tor_stations.to_csv('tor_stations.csv', index=False)